<a href="https://colab.research.google.com/github/vfrantc/dehaze_detect/blob/main/evaluate_detection_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

Mon Nov  8 13:36:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


# RUN DETECTOR ON UNPROCESSED IMAGES

In [3]:
!pip install mmcv-full
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -e .
!pip install Pillow==7.0.0

     |████████████████████████████████| 389 kB 10.0 MB/s 
     |████████████████████████████████| 185 kB 67.8 MB/s 
  Created wheel for mmcv-full: filename=mmcv_full-1.3.16-cp37-cp37m-linux_x86_64.whl size=43452581 sha256=cf80b1bc2ab8febf8741803d5b8f19d5f9f9089fe7e8ef45a410d25791abcf8d
  Stored in directory: /root/.cache/pip/wheels/ea/0a/1e/95428dfef725d66e6693b61180b928c293489b6d13df3ae0e2
Successfully built mmcv-full
Cloning into 'mmdetection'...
remote: Enumerating objects: 21571, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 21571 (delta 3), reused 3 (delta 1), pack-reused 21555
Receiving objects: 100% (21571/21571), 25.06 MiB | 33.50 MiB/s, done.
Resolving deltas: 100% (15142/15142), done.
/content/mmdetection
Obtaining file:///content/mmdetection
  Created wheel for terminaltables: filename=terminaltables-3.1.0-py3-none-any.whl size=15354 sha256=494e2893447b56cf2828fc73d9bb8b5709c5400e1ae11d4a7f32701e66eee369
  

In [4]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmdetection/v2.0/scnet/scnet_x101_64x4d_fpn_20e_coco/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth \
      -O checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth

--2021-11-08 12:36:26--  https://download.openmmlab.com/mmdetection/v2.0/scnet/scnet_x101_64x4d_fpn_20e_coco/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.88.36.59
Connecting to download.openmmlab.com (download.openmmlab.com)|47.88.36.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 611860651 (584M) [application/octet-stream]
Saving to: ‘checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth’

checkpoints/scnet_x 100%[===================>] 583.52M  8.64MB/s    in 68s     

2021-11-08 12:37:35 (8.60 MB/s) - ‘checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth’ saved [611860651/611860651]



In [6]:
%cd mmdetection

/content/mmdetection


In [5]:
from glob import glob
import os

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.datasets import build_dataloader
from mmdet.apis import single_gpu_test
from mmcv.parallel import MMDataParallel
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

config = 'configs/scnet/scnet_x101_64x4d_fpn_20e_coco.py'
checkpoint = 'checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth'
model = init_detector(config, checkpoint, device='cuda:0')
model = MMDataParallel(model, device_ids=[0])

Use load_from_local loader


In [6]:
def create_dataset(folder):
  cfg = Config.fromfile('./configs/scnet/scnet_x101_64x4d_fpn_20e_coco.py')
  cfg.dataset_type = 'CocoDataset'
  PREFIX = folder
  cfg.data.test.img_prefix = folder
  cfg.data.test.ann_file = os.path.join(PREFIX, 'new_test.json')
  cfg.data.test.type = 'CocoDataset'
  dataset = build_dataset(cfg.data.test)
  #dataset.CLASSES = classes
  #dataset.cat_ids = list(range(len(dataset.CLASSES)))
  data_loader = build_dataloader(dataset, samples_per_gpu=1, workers_per_gpu=2, dist=False, shuffle=False)
  return dataset, data_loader

In [7]:
!cp /content/drive/MyDrive/hazy_detect/out_gca.zip .
!cp /content/drive/MyDrive/hazy_detect/dcp_coco.zip . 
!cp /content/drive/MyDrive/hazy_detect/out_refined.tar.gz .
!cp /content/drive/MyDrive/hazy_detect/dehazenet_coco.zip .
!cp /content/drive/MyDrive/hazy_detect/ide_coco.zip .

In [8]:
!unzip -q out_gca.zip
!unzip -q dcp_coco.zip
!tar xzvf out_refined.tar.gz
!unzip -q dehazenet_coco.zip
!unzip -q ide_coco.zip

out_refined/
out_refined/new_test.json


### gca

In [9]:
!cp /content/drive/MyDrive/new_test.json out_gca

In [ ]:
method = 'out_gca'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
outputs = single_gpu_test(model, data_loader, True, f'./out_{method}', 0.3)


loading annotations into memory...
Done (t=0.50s)
creating index...
index created!
[                                                  ] 0/4320, elapsed: 0s, ETA:

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[>>>>>>>>>>>>>>>>>>        ] 3104/4320, 1.5 task/s, elapsed: 2100s, ETA:   823s

In [ ]:
# post-process the detection to make sure that we use only the correct ones

In [ ]:
dataset.cat_ids.append(91)
metric = dataset.evaluate(outputs, metric='bbox')

In [1]:
!ls

drive  mmdetection  sample_data


In [ ]:
!tar czf detected_gca.tar.gz out_out_gca/
!cp detected_gca.tar.gz /content/drive/MyDrive/new_detect

### dcp


In [ ]:
method = 'dcp_coco'
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, True, f'./out_{method}', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

In [ ]:
!tar czf detected_dcp.tar.gz out_dcp_coco/
!cp -r detected_dcp.tar.gz /content/drive/MyDrive/new_detect

### Refined

In [ ]:
!cp /content/drive/MyDrive/new_test.json out_refined

In [ ]:
!ls out_refined/JPEGImages

In [ ]:
method = 'out_refined'
!rm -rf out_out_refined
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, True, f'./out_{method}', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

In [ ]:
!tar czf detected_refined.tar.gz out_out_refined/
!cp -r detected_refined.tar.gz /content/drive/MyDrive/new_detect

# Dehazenet

In [ ]:
method = 'dehazenet_coco'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, True, f'./out_{method}', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

In [ ]:
!tar czf detected_dehazenet.tar out_dehazenet/
!cp -r detected_dehazenet.zip /content/drive/MyDrive/new_detect

# IDE

In [ ]:
method = 'dide_coco'
!cp /content/drive/MyDrive/new_test.json dide_coco
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, True, f'./out_{method}', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

In [ ]:
!tar czf detected_ide.tar.gz out_ide_coco/
!cp -r detected_ide.tar.gz /content/drive/MyDrive/new_detect

# RTTS

In [ ]:
!rm -rf rtts_coco
!cp /content/mmdetection/drive/MyDrive/haze/data/RTTS.zip .
!unzip -q RTTS.zip
!mv RTTS rtts_coco 
!cp /content/mmdetection/drive/MyDrive/new_test.json rtts_coco

In [ ]:
method = 'rtts_coco'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, True, f'./out_{method}', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

In [ ]:
!tar czf detected_rtts_coco.tar.gz out_rtts_coco/
!cp -r detected_rtts_coco.tar.gz /content/drive/MyDrive/new_detect